In [153]:
import numpy as np
import copy
from math import log2
from collections import defaultdict
import time

# class Node:
#     def __init__(self, emitter=None, split=None, roomlabel=None):
#         self.right = None
#         self.left = None
#         self.emitter = emitter
#         self.split = split
#         self.roomlabel = roomlabel

In [154]:
def entropy(roomData):
    H=0.0
    i=0
    scoreboard = {1:0, 2:0, 3:0, 4:0}
    size = len(roomData)
    while i<size:
        if roomData[i][-1] == 1:
            scoreboard[1]+=1
        elif roomData[i][-1] == 2:
            scoreboard[2]+=1
        elif roomData[i][-1] == 3:
            scoreboard[3]+=1
        elif roomData[i][-1] == 4:
            scoreboard[4]+=1
        i+=1;
    for i in (1,2,3,4):
        if scoreboard[i] > 0:
            H += ((-scoreboard[i])/size)*(log2(scoreboard[i]/size))
    return H

In [155]:
def InfoGain(All, Left, Right):
    H_SAll=entropy(All)
    Total=len(Left)+len(Right)
    Remainder=((len(Left)/Total)*entropy(Left))+((len(Right)/Total)*entropy(Right))
    Gains=H_SAll-Remainder
    return Gains

In [156]:
def FIND_SPLIT(d):
    emitter = 0
    value = 0
    max_info_gain = 0
    for x in range(len(d[0])-1):
        #d.sort(d,key = lambda y: y[x])
        ds = np.array(sorted(d, key = lambda y: y[x], reverse=True))
        #print(ds)
        for r in range(len(ds)):
            
            split_point = ds[r][x]
            if(r!=len(ds)-1 and ds[r+1][x] == split_point):
                continue
            info_gain = InfoGain(ds, ds[:r+1],ds[r+1:])
            
            if(info_gain > max_info_gain):
                emitter = x
                value = split_point
                max_info_gain = info_gain
            
            
        
    return emitter, value

In [157]:
def split(clean_rows):
    Left = []
    Right = []
    em, val = FIND_SPLIT(clean_rows) #splitvalue[0] = emitter and splitvalue[1] = value under that column
    
    for i in range(len(clean_rows)):
        if(clean_rows[i][em] >= val):
            Left.append(clean_rows[i])
        
        else:
            Right.append(clean_rows[i])
        
    
    
    return np.array(Left),np.array(Right),em,val

In [213]:
def tree_learning(training_data, depth):
    if(entropy(training_data) == 0 and len(training_data) != 0):
        count = len(training_data)
        label = training_data[0][-1]
        leaf = {'emitter':-1, 'value':-1, 'room': label, 'right':-1, 'left':-1, 'count': count}
        return leaf, depth
    else:
        ldata, rdata, em, val = split(training_data)
        root = {'emitter': em, 'value': val, 'room': -1, 'right':-1, 'left':-1, 'count': -1}
        root['left'], l_depth = tree_learning(ldata, depth+1)
        root['right'], r_depth = tree_learning(rdata, depth+1)
        return root, max(l_depth, r_depth)

In [159]:
# def tree_learning2(training_data):
#     if(entropy(training_data) == 0):
#         label = training_data[0][-1]
#         leaf = Node(roomlabel=label)
#         return leaf
#     else:
#         ldata, rdata, em, val = split(training_data)
#         root = Node(emitter=em, split=val)
#         root.left = tree_learning2(ldata)
#         root.right = tree_learning2(rdata)
#         return root

In [160]:
clean_rows = np.loadtxt("clean_dataset.txt")
noisy_rows = np.loadtxt("noisy_dataset.txt")
print(clean_rows)

[[-64. -56. -61. ... -82. -81.   1.]
 [-68. -57. -61. ... -85. -85.   1.]
 [-63. -60. -60. ... -85. -84.   1.]
 ...
 [-62. -59. -46. ... -87. -88.   4.]
 [-62. -58. -52. ... -90. -85.   4.]
 [-59. -50. -45. ... -88. -87.   4.]]


In [161]:
t, d = tree_learning(noisy_rows, 0)
#print(t)

In [162]:
def cross_validation(dataset):

#shuffle 
    np.random.shuffle(dataset)
#take a set
    fold_size = int(len(dataset)/10)
    test = dataset[0:fold_size]
    validation = []
    classes = set({})
    running_cm = np.zeros((4,4))
    
    for i in range(len(dataset)):
        classes.add(dataset[i][-1])
    
    for i in range(10):
        test = dataset[0+fold_size*i:fold_size*(i+1)]
        training = np.vstack((dataset[0:0+fold_size*i],dataset[fold_size*(i+1):]))
        root, depth = tree_learning(training, 0)
        
        confusion_matrix, tree_accuracy = evaluate(test, root)        
        running_cm += confusion_matrix
        
        #pruned_tree = prune(root, validation set)
        
        
    
    avg_cm = running_cm/10
    
    return accuracy(avg_cm)
    
    
        
#cycle validation

In [163]:
def evaluate(test, root): #why we do like this we dont need to
    #recurse on evaluate we can make a function to just return the leaf value and comapre in the for loop and form
    #the whole matrix here, the other function will just return our value
    
    arr = []
    
    for row in test:
        prediction = return_result(row, root)
        arr.append([row[-1],prediction])
        
    #print(arr)
    cm = confuse_matrix(arr)
    tree_accuracy = accuracy(cm)
    
    return cm, tree_accuracy

In [164]:
def confuse_matrix(arr): 
    
    cm = np.zeros((4,4))
    
    for i in range (1,5):
        for pair in arr:
            if(i == pair[0] and pair[0] == pair[1]):
                    cm[i-1][i-1]+=1
            if(i == pair[0] and pair[1] != i):
                    cm[i-1][int(pair[1])-1]+=1
    return cm
    
#confuse_matrix([[1, 1], [2, 2], [3,3], [1,2], [1,3], [3,1], [4,2]])

In [165]:
def return_result(row, root):
    
    if(root['room']!= -1):
        return root['room']
    else:
        if(row[root['emitter']]>=root['value']):
            return return_result(row, root['left'])
        else:
            return return_result(row, root['right'])

In [232]:
def accuracy(cm):
#     sum = 0
#     trace = 0
#     for i in range(len(cm)):
#         trace+=cm[i][i]
#     for i in cm:
#         sum+=i.sum()
    if(np.sum(cm) == 0):
        return 0
    return np.trace(cm)/np.sum(cm)

In [167]:
#-----------------------------------------#
testing = np.loadtxt("clean_dataset.txt")
st=time.time()
print(cross_validation(testing))
et=time.time()
print('Runtime: ', et-st)
#-----------------------------------------#

0.973
Runtime:  71.2025740146637


In [168]:
def precision(cm):
    arr = []
    cols = cm.sum(axis=0)
    
    for i in range(len(cm)):
        arr.append(cm[i][i]/cols[i])
    
    return arr

In [169]:
def recall(cm):
    arr = []
    rows = cm.sum(axis=1)
   
    for i in range(len(cm)):
        arr.append(cm[i][i]/rows[i])
    
    return arr

In [54]:
a = np.arange(1,17).reshape(4,4)
#print(a)
print(recall(a))
print(precision(a))

[0.1, 0.23076923076923078, 0.2619047619047619, 0.27586206896551724]
[0.03571428571428571, 0.1875, 0.3055555555555556, 0.4]


In [170]:
def f1(recall, precision):
    arr = []
    for i in range(len(recall)):
        arr.append(2*r[i]*p[i]/r[i]+p[i])
    
    return arr
    

In [56]:
r = [0.1, 0.23076923076923078, 0.2619047619047619, 0.27586206896551724]
p = [0.03571428571428571, 0.1875, 0.3055555555555556, 0.4]
print(f1(r,p))

[0.10714285714285714, 0.5625, 0.9166666666666667, 1.2000000000000002]


In [215]:
def prune(root, validation_set, start):
    
    #print(type(root))
    #print('Left Room', root['left']['room'])
    #print('Right Room', root['right']['room'])

    
    
    if ((root['left']['emitter'] == -1) and (root['right']['emitter'] == -1)):
        
        old = [root['emitter'], root['value'], root['room'], root['right'], root['left']]

        cm, old_accuracy = evaluate(validation_set, start)
        
        #print('NumLeft: ', numClasses[root['left']['room']])
        
        if(root['left']['count'] > root['right']['count']):
            root['room'] = root['left']['room']
        else:
            root['room'] = root['right']['room']

        root['left'] = -1
        root['right'] = -1
        root['emitter'] = -1
        root['value'] = -1
        
        cm, new_accuracy = evaluate(validation_set, start)
        
        if new_accuracy < old_accuracy:
            root['emitter'] = old[0]
            root['value'] = old[1]
            root['room'] = old[2]
            root['right'] = old[3]
            root['left'] = old[4]
        
               
    else:
        
        if(root['left']['emitter'] != -1):
            start = prune(root['left'], validation_set, start)
        if(root['right']['emitter'] != -1):
            start = prune(root['right'], validation_set, start)
    
    return start

In [234]:
def cross_validation_prune(dataset):

#shuffle 
    np.random.shuffle(dataset)
#take a set

    fold_size = int(len(dataset)/10)
    classes = set({})
    running_cm = np.zeros((4,4))
    running_cm_pr = np.zeros((4,4))
    
    
    for i in range(len(dataset)):
        classes.add(dataset[i][-1])
    
    for i in range(10):
        
        #numclasses = {1:0, 2:0, 3:0, 4:0}
        test = dataset[fold_size*i:fold_size*(i+1)]
        validation = dataset[fold_size*(i+1)+1:fold_size*(i+2)+1]
        training = np.vstack((dataset[0:fold_size*i],dataset[fold_size*(i+2)+1:]))
        root, depth = tree_learning(training, 0)
        
        
        confusion_matrix, tree_accuracy = evaluate(test, root)        
        running_cm += confusion_matrix
        
        #========================================================================================#
        
#         for j in training:
#             numclasses[j[-1]]+=1
        
        #print(numclasses)
        pruned_tree = prune(root, validation, root)
#         print('Depth after:', findDepth(pruned_tree))
        
        cm_prune, pr_acc = evaluate(test, pruned_tree)
        running_cm_pr += cm_prune
        
#         print(depth, " vs ", findDepth(pruned_tree))
#         print(tree_accuracy, " vs ", pr_acc)

        #========================================================================================#
    
    avg_cm = running_cm/10
    avg_cm_pr = running_cm_pr/10
    
#     print("Before prune accuracy:", accuracy(avg_cm))
#     print("After prune accuracy:", accuracy(avg_cm_pr))
    
    return accuracy(avg_cm)
    
    
        
#cycle validation

In [236]:
#-----------------------------------------#
testing = np.loadtxt("noisy_dataset.txt")
st=time.time()
print(cross_validation_prune(testing))
et=time.time()
print('Runtime: ', et-st)
#-----------------------------------------#

0.8044999999999999
Runtime:  108.35124897956848


In [203]:
def findDepth(node): 
    
    lDepth = 0
    rDepth = 0
    
    if node is None: 
        return 0 ;  
  
    else : 
  
        # Compute the depth of each subtree 
        
        if(node['left']['emitter'] != -1):
            lDepth = findDepth(node['left']) 
        if(node['right']['emitter'] != -1):
            rDepth = findDepth(node['right']) 
  
        # Use the larger one 
        if (lDepth > rDepth): 
            return lDepth+1
        else: 
            return rDepth+1